LLM控制面板

In [1]:
import sys
sys.path.append("../")
from model import *
from IPython.display import display

# import os
# os.environ['CUDA_VISIBLE_DEVICES'] = '6'

vicuna_chat_template = "A chat between a curious user and an artificial intelligence assistant. The assistant gives helpful, detailed, and polite answers to the user's questions.\n\n##USER:\n{}\n\n##ASSISTANT:\n"
internlm_chat_template = "<|User|>:{}<eoh>\n<|Bot|>:"

panel = LLMPanel(
    model_list=[
        ("gpt2", "/home/cs/yangyuchen/guoyiqiu/my_models/gpt2"),
        ("llama_7b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-7b-hf"),
        ("llama_13b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-13b"),
        ("llama-2-7b-chat", "/home/cs/yangyuchen/yushengliao/Medical_LLM/llama-2-7b-chat-hugging"),
        ("llama-2-13b-chat", "/home/cs/yangyuchen/guoyiqiu/my_models/Llama-2-13b-chat-ms"),
        ("vicuna_7b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/vicuna-7b/"),
        ("internlm-chat-7b", "/home/cs/yangyuchen/yushengliao/Medical_LLM/internlm-chat-7b"),
        ("internlm-chat-20b", "/home/cs/yangyuchen/guoyiqiu/my_models/internlm-chat-20b")
        ],
    chat_template=internlm_chat_template)
display(panel)
panel.setup_btn.click()
panel.device_tbtn.value = 'cuda'

LLMPanel(children=(HBox(children=(Dropdown(description='Model:', options=(('gpt2', '/home/cs/yangyuchen/guoyiq…

Using pad_token, but it is not set yet.


Everything is ready. Time cost: 3.82s


FlowVisualizer

In [2]:
tab1,tab2 = panel.mt.vis_sentence('hello world')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
